# 전체 코드 실행 후 테스트 코드를 실행해주세요.

In [9]:
import torch
from torch import nn
import torch.nn.functional as F
import gluonnlp as nlp
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
import pandas as pd
import numpy as np
import rhinoMorph
# Loading en_vocab_list
with open("/content/drive/MyDrive/Ai_test/1000common_list_en.txt", "r") as file:
    lines = file.read().split(',')
# Loading Rihno
rn = rhinoMorph.startRhino()
# Setting Device
device = torch.device("cpu")
# Loading Kobert Vocab
_, vocab = get_pytorch_kobert_model()

filepath:  /usr/local/lib/python3.6/dist-packages
classpath:  /usr/local/lib/python3.6/dist-packages/rhinoMorph/lib/rhino.jar
RHINO started!
[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [10]:
class BERTClassifier(nn.Module):
    def __init__(self, bert, hidden_size = 768, num_classes=9, dr_rate=None, params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [18]:
# Loading Model
model = torch.load('/content/drive/MyDrive/Ai_test/model_KoBert_RhinoDataRefine.pt', map_location=device)
model.eval()
print('Model Loaded')

Model Loaded


In [23]:
def del_unimportant_n(sentence, lines, rn):
  sentence = sentence.replace('\r','')
  sentence = sentence.replace('\n','')
  sentence = sentence.replace('\([^)]*\)', '')
  sentence = str.upper(sentence)
  sentence = sentence.replace("[^가-힣A-Z0-9=%\. ]","")
  del_dic = {}
  n_list = rhinoMorph.onlyMorph_list(rn, sentence, pos=['SL'])
  for n in n_list:
    if len(n)>1:
      if n not in lines:
        del_dic[n] = ''
  if len(del_dic)!=0:
    sentence = pd.Series(sentence)
    new_sentence = sentence.replace(del_dic, regex=True)
  else:
    new_sentence = pd.Series(sentence)
  new_sentence = new_sentence.replace({' +':' '}, regex=True)
  return new_sentence[0]

def BERT_change(sentence) :
    sentence = del_unimportant_n(sentence, lines, rn)
    tokenizer= nlp.data.BERTSPTokenizer(get_tokenizer(), vocab, lower=False)
    transform = nlp.data.BERTSentenceTransform(tokenizer, max_seq_length = 128, pad = True, pair = False)
    sentence_val = transform([sentence])
    inputs = torch.tensor(sentence_val[0], dtype=torch.long)
    inputs = inputs.unsqueeze(0)
    lens = torch.tensor(sentence_val[1])
    lens = lens.unsqueeze(0)
    masks = torch.tensor(sentence_val[2])
    return inputs, lens, masks

def test_sentences(sentences):
    inputs, lens, masks = BERT_change(sentences)
    b_input_ids = inputs.to(device)
    b_lens = lens.to(device)
    b_masks = masks.to(device)
    with torch.no_grad():     
        outputs = model(b_input_ids, valid_length = b_lens, segment_ids = b_masks)
    logits = outputs[0]
    logits = logits.detach().cpu().numpy()
    tagging = np.argmax(logits)
    dicts_data = {0 : '가설 설정', 1 : '기술 정의', 2 : '대상 데이터', 3 : '데이터처리',
                  4 : '문제 정의', 5 : '성능/효과', 6 : '이론/모형', 7 : '제안 방법', 8 : '후속연구'}
    tagging = dicts_data[tagging]
    return tagging

# 테스트 코드

In [24]:
print("파일명을 입력하세요 (xlsx 파일)")
x = input()
data_xlsx = pd.read_excel(("/content/drive/MyDrive/Ai_test/"+ x + ".xlsx"))
data_xlsx['결과'] = 0
for i, j in enumerate(data_xlsx['문장']) :
    data_xlsx.loc[i, '결과'] = test_sentences(j)
count = 0
for i in range(len(data_xlsx['문장'])):
    if data_xlsx.loc[i, '태그'] == data_xlsx.loc[i, '결과'] :
      count += 1
print("-"*20)
for i in data_xlsx['태그'].unique():
    x = data_xlsx[data_xlsx['태그'] == i]
    y = x[x['결과'] == i]
    print("{} : {}%".format(i, round(len(y)/len(x)*100, 2)))
    print("-"*20)
print("종합 : {}%".format(round(count / len(data_xlsx['문장']) * 100, 2)))

파일명을 입력하세요 (xlsx 파일)
test
using cached model
using cached model
using cached model
using cached model
using cached model
using cached model
using cached model
using cached model
using cached model
using cached model
using cached model
using cached model
using cached model
using cached model
using cached model
using cached model
using cached model
using cached model
using cached model
using cached model
using cached model
using cached model
using cached model
using cached model
using cached model
using cached model
using cached model
using cached model
using cached model
using cached model
using cached model
using cached model
using cached model
using cached model
using cached model
using cached model
using cached model
using cached model
using cached model
using cached model
using cached model
using cached model
using cached model
using cached model
using cached model
using cached model
using cached model
using cached model
using cached model
using cached model
using cached model
using

In [25]:
# Check tagging Results
data_xlsx

,태그,문장,결과
0,가설 설정,커피찌꺼기의 성분에는 최 등 2)의 연구에서 밝힌바와 같이 섬유성분이 46.6-51...,가설 설정
1,가설 설정,"중화기를 통과하는 공기 유량이 0.3 L/min인 경우, 입자가 평형 대전량 분포를...",가설 설정
2,가설 설정,그렇기 때문에 주거의 기능은 디지털 시대 에 발 맞추어 LED의 다양한 조명연출 장...,가설 설정
3,가설 설정,"이렇듯 상당한 부분의 소비전력을 조명전력으로 사용하고 있는 실정이며, 선진국으로 갈...",가설 설정
4,가설 설정,"온실가스 저감량 도출은 2022년을 기준으로 1,490MW의 태양광발전 의무량이 설...",가설 설정
...,...,...,...
99,후속연구,"하지만 다시 의예과 선발 체제로 전환되기로 한 점, 의학과 4년 교육과정의 개발이 ...",후속연구
100,후속연구,본 연구에서는 20명의 거북목 증후군 환자들을 대상으로 하여 결과를 일반화하기에는 ...,후속연구
101,후속연구,본 논문에서는 간단한 구현 방안에 대해서만 제안하고 상세 구현 방안은 추후 연구를 ...,후속연구
102,후속연구,"향후,도서추천 목록에 대한 정확한 만족도 조사를 위한 별도의 방법이 마련되어야 하며...",후속연구
